In [1]:
%load_ext lab_black
%cd ../../..

/home/shim/cev/dl/log-analytics


In [2]:
import torch
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from collections import defaultdict
import sys

In [3]:
distdeck = torch.load(
    "results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_5-distdeck.pth"
)

In [6]:
deck2 = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_5-deck2.pth")

In [7]:
df = pd.read_csv("data/ori/test.csv")

In [8]:
distdeck["dist"].shape, distdeck["level"].shape

(torch.Size([1095951, 4]), torch.Size([1095951, 4]))

In [9]:
distdeck.keys(), deck2.keys()

(dict_keys(['dist', 'level', 'ids']),
 dict_keys(['feat', 'otext', 'fclevel', 'ids']))

In [10]:
total_len = 1418916

In [99]:
def policy(dists, tlevels, fclevel):
    if fclevel in [6, 4, 2] or tlevels[0].item() in [6, 4, 2]:
        return fclevel.item()
    if (tlevels == 5).all():
        return 5 if dists[0] < 1.2 else 7
    if (tlevels == 3).all():
        return 3 if dists[0] < 1.2 else 7
    if dists[0] < 1.0:
        # return tlevels[0].item()
        return fclevel.item()
    return 7

In [100]:
out_dists = [None for _ in range(total_len)]
out_levels = [None for _ in range(total_len)]
out_fclevels = [None for _ in range(total_len)]
N = distdeck["dist"].size(0)
with tqdm(total=N, ncols=100, file=sys.stdout) as t:
    for i in range(N):
        dists = distdeck["dist"][i]
        levels = distdeck["level"][i]
        fclevel = deck2["fclevel"][i]
        out_level = policy(dists, levels, fclevel)
        ids = deck2["ids"][i]
        for j in ids:
            out_levels[j - 1000000] = out_level
            out_dists[j - 1000000] = dists
            out_fclevels[j - 1000000] = fclevel
        t.update()

100%|██████████████████████████████████████████████████| 1095951/1095951 [01:20<00:00, 13650.38it/s]


In [101]:
out_levels = np.array(out_levels)

In [102]:
# 각 레벨별 개수 0.5이하일 때 fclevel을 쓰기로
for i in range(8):
    cnt = (out_levels == i).sum()
    print(i, ":", cnt, f"{cnt / len(out_levels)*100:.2f}%")

0 : 1003875 70.75%
1 : 395022 27.84%
2 : 42 0.00%
3 : 12909 0.91%
4 : 34 0.00%
5 : 6361 0.45%
6 : 29 0.00%
7 : 644 0.05%


In [17]:
train_df = pd.read_csv("data/ori/train.csv")

In [88]:
(out_levels == 6).nonzero()[0][:100]

array([  35574,   64268,   73308,   86431,   92457,  147553,  175492,
        179014,  210545,  240872,  340297,  355854,  374455,  451522,
        484782,  499683,  507877,  529914,  624698,  633202,  647782,
        652696,  714497,  848132,  881660, 1113383, 1125342, 1169189,
       1394234])

In [89]:
df.full_log[563330]

'Sep 18 10:31:30 localhost useradd[5205]: new user: name=kibana, UID=981, GID=977, home=/home/kibana, shell=/sbin/nologin'

In [62]:
def find(text):
    for i, full_log in enumerate(train_df.full_log):
        if full_log.find(text) != -1:
            return i
    return -1


def find_all(text):
    l = []
    for i, full_log in enumerate(train_df.full_log):
        if full_log.find(text) != -1:
            l.append(i)
    return l

In [97]:
find_all("localhost useradd")

[221847]

In [98]:
i = 221847
train_df.level[i], train_df.full_log[i]

(6, 'Jan 29 11:28:59 localhost useradd[88679]: new group: name=test, GID=1001')

---

In [68]:
out_ids = list(range(1000000, 1000000 + len(out_levels)))

In [69]:
out_df = {"id": out_ids, "level": out_levels}

In [70]:
out_df = pd.DataFrame(out_df)

In [71]:
out_df

,id,level
0,1000000,0
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,0
1418913,2418913,1
1418914,2418914,0


In [72]:
out_df.to_csv(
    "results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_5-out_ver1.csv", index=False
)